In [4]:
import numpy as np
from astropy.table import Table
import thecannon as tc

In [ ]:
#paths
table_path = '/home/mrl2068/Desktop/CEMP/'
spec_path = '/home/mrl2068/Desktop/CEMP/' #assumes spec are in directory 'apogee' inside this path
fig_path = '/home/mrl2968/Desktop/CEMP/DR3_feh/'

In [5]:
t = Table.read(table_path+'synth_apogee_0.fits')
t = t[t['synth']]


In [ ]:
#loading in spec with max normalization


for i in range(len(t)):
    Teff = t['Teff'][i] ; logg = t['logg'][i] ; feh = t['feh'][i] ;  a = t['A_v'][i] ; s = t['SNR'][i]
    filename = '%s_g%s_f%s_a%s_n%s.txt'%(str(Teff),str(logg),str(feh),str(a),str(s))
    try:
        w, fbp, ebp = np.loadtxt(spec_path+'apogee/Bp_'+filename,usecols=(0,1,2),skiprows=1,unpack=True)
        wr, frp, erp = np.loadtxt(spec_path+'/apogee/Rp_'+filename,usecols=(0,1,2),skiprows=1,unpack=True)
    except:  
        try:
            formatter = "{0:.12g}"
            Teff2 = formatter.format(Teff)
            logg2 = formatter.format(logg)
            feh2 = formatter.format(feh)
            s2 = formatter.format(s)
            a2 = formatter.format(a)
            filename = '%s_g%s_f%s_a%s_n%s.txt'%(str(Teff2),str(logg2),str(feh2),str(a2),str(s2))
            w, fbp, ebp = np.loadtxt(spec_path+'apogee/Bp_'+filename,usecols=(0,1,2),skiprows=1,unpack=True)
            wr, frp, erp = np.loadtxt(spec_path+'apogee/Rp_'+filename,usecols=(0,1,2),skiprows=1,unpack=True)
        except:
            try:
                formatter = "{0:.12g}"
                logg2 = formatter.format(logg)
                feh2 = formatter.format(feh)
                s2 = formatter.format(s)
                a2 = formatter.format(a)
                filename = '%s_g%s_f%s_a%s_n%s.txt'%(str(Teff),str(logg2),str(feh2),str(a2),str(s2))
                w, fbp, ebp = np.loadtxt(spec_path+'apogee/Bp_'+filename,usecols=(0,1,2),skiprows=1,unpack=True)
                wr, frp, erp = np.loadtxt(spec_path+'apogee/Rp_'+filename,usecols=(0,1,2),skiprows=1,unpack=True)
            except:
                formatter = "{0:.12g}"
                Teff2 = formatter.format(Teff)
                logg2 = formatter.format(logg)
                feh2 = formatter.format(feh)
                a2 = formatter.format(a)
                s = "{0:.1f}".format(s)
                filename = '%s_g%s_f%s_a%s_n%s.txt'%(str(Teff2),str(logg2),str(feh2),str(a2),str(s))
                w, fbp, ebp = np.loadtxt(spec_path+'apogee/Bp_'+filename,usecols=(0,1,2),skiprows=1,unpack=True)
                wr, frp, erp = np.loadtxt(spec_path+'apogee/Rp_'+filename,usecols=(0,1,2),skiprows=1,unpack=True)
    w = w[~np.isnan(ebp)]
    wr = wr[~np.isnan(erp)]
    fbp = fbp[~np.isnan(ebp)]
    ebp = ebp[~np.isnan(ebp)]
    frp = frp[~np.isnan(erp)]
    erp = erp[~np.isnan(erp)]
   
    if i == 0:
        xs = np.zeros((len(t),len(np.concatenate([fbp,frp]))))
        xerrs = np.zeros((len(t),len(np.concatenate([fbp,frp]))))
        
    f = np.concatenate([fbp,frp])
    errs = 1.0/np.sqrt(np.concatenate([ebp,erp]))
    
    xs[i,:] = f/np.max(f)
    

    xerrs[i,:] = errs/np.max(errs)
    wf = np.concatenate([w,wr])
    

In [ ]:
#making training and testing sets

train_inds = np.random.choice(range(len(t)),int(.6*len(t)),replace=False)
test_inds = set(range(len(t)))- set(train_inds)


train_xs = xs[train_inds]
train_errs = xerrs[train_inds]

test_xs = xs[test_inds]
test_errs = xerrs[test_inds]

train_t = t[train_inds]

In [ ]:
# set up model
vectorizer = tc.vectorizer.PolynomialVectorizer(("Teff", "logg", "feh","A_v"), 2)

model = tc.CannonModel(train_t, train_xs, train_errs,
                       vectorizer=vectorizer, dispersion=wf, threads=-1)

# Train the model!
model.train()


In [ ]:
#infer for train
test_labels, cov, metadata = model.test(test_xs, test_errs)

fig_comparison = tc.plot.one_to_one(model, test_labels, latex_label_names=[r"T_{\rm eff}",r"\log{g}",r"[{\rm Fe}/{\rm H}]",r"{\rm A_V}"])
fig_comparison.savefig(fig_path+"one-to-one.png", dpi=300)
